In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

## Define Sample Data

In [2]:
corpus = ["apple banana fruit", "banana apple fruit", "banana fruit apple",
                 "dog cat animal", "cat animal dog", "cat dog animal"]

In [3]:
corpus = [sent.split(" ") for sent in corpus]
corpus

[['apple', 'banana', 'fruit'],
 ['banana', 'apple', 'fruit'],
 ['banana', 'fruit', 'apple'],
 ['dog', 'cat', 'animal'],
 ['cat', 'animal', 'dog'],
 ['cat', 'dog', 'animal']]

In [4]:
#extracting unique words
flatten = lambda l: [item for sublist in l for item in sublist]
vocab = list(set(flatten(corpus)))
vocab

['cat', 'animal', 'apple', 'banana', 'fruit', 'dog']

In [5]:
#appending number for each word in the set
word2index = {w: i for i, w in enumerate(vocab)}
print(word2index)

{'cat': 0, 'animal': 1, 'apple': 2, 'banana': 3, 'fruit': 4, 'dog': 5}


In [6]:
#vocab size
voc_size = len(vocab)
print(voc_size)

6


In [8]:
#append UNK
vocab.append('<UNK>')

In [9]:
vocab

['cat', 'animal', 'apple', 'banana', 'fruit', 'dog', '<UNK>', '<UNK>']

In [10]:
#defining a method to get the word through index
index2word = {v:k for k, v in word2index.items()} 